Name: Sudhina D

Batch: R3

Date: 10-03-2023

Experiment Name: Text Autocompletion



Contents

    Dataset download
    Import libraries
    Load the dataset
    VISUALAZING THE DATASET
    DATASET Preprocessing
    RNN model
    Train the model
    Predictiom
    Evaluation




Dataset download

The data contains information about the comments made on the articles published in New York Times in Jan-May 2017 and Jan-April 2018. The month-wise data is given in two csv files - one each for the articles on which comments were made and for the comments themselves. The csv files for comments contain over 2 million comments in total with 34 features and those for articles contain 16 features about more than 9,000 articles.

LINK - https://www.kaggle.com/datasets/aashita/nyt-comments?resource=download


import drive

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


Import libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
!apt-get -qq install -y graphviz && pip install pydot
!pip install cartopy
import cartopy
import pydot
import libarchive
# keras module for building LSTM 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
import string, os 

import warnings
warnings.filterwarnings("ignore")


Load the dataset  

VISUALAZING THE DATASET

In [ ]:
# Checking out one dataset

testing_ar = pd.read_csv('/content/gdrive/MyDrive/DNNSP LAB/text.csv/ArticlesApril2017.csv')
testing_ar.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


In [ ]:
# Checking out the comments
testing_cm = pd.read_csv('/content/gdrive/MyDrive/DNNSP LAB/text.csv/CommentsApril2017.csv')
testing_cm.head()

,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,...,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,This project makes me happy to be a 30+ year T...,22022598.0,22022598,<br/>,comment,1.491237e+09,1,False,0.0,...,"Riverside, CA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
1,1491188619,Stunning photos and reportage. Infuriating tha...,22017350.0,22017350,NaN,comment,1.491180e+09,1,False,0.0,...,<br/>,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
2,1491188617,Brilliant work from conception to execution. I...,22017334.0,22017334,<br/>,comment,1.491179e+09,1,False,0.0,...,Raleigh NC,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
3,1491167820,NYT reporters should provide a contributor's l...,22015913.0,22015913,<br/>,comment,1.491150e+09,1,False,0.0,...,"Missouri, USA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News
4,1491167815,Could only have been done in print. Stunning.,22015466.0,22015466,<br/>,comment,1.491147e+09,1,False,0.0,...,"Tucson, Arizona",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716.0,2,News


In [ ]:
# File path for first file:
# ../input/nyt-comments/ArticlesApril2017.csv
# Some of the headlines have - Unknown as a null value
# We will extract all headlines
working_dir = '/content/gdrive/MyDrive/DNNSP LAB/text.csv/'
headlines = []

for filename in os.listdir(working_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(working_dir + filename)
        headlines.extend(list(article_df.headline.values))

headlines = [x for x in headlines if x!="Unknown"]


In [ ]:
len(headlines)

8603

Text cleaning

In [ ]:
sample = headlines[:10]
sample

['Finding an Expansive View  of a Forgotten People in Niger',
 'And Now,  the Dreaded Trump Curse',
 'Venezuela’s Descent Into Dictatorship',
 'Stain Permeates Basketball Blue Blood',
 'Taking Things for Granted',
 'The Caged Beast Awakens',
 'An Ever-Unfolding Story',
 'O’Reilly Thrives as Settlements Add Up',
 'Mouse Infestation',
 'Divide in G.O.P. Now Threatens Trump Tax Plan']

In [ ]:
# Basic iteration over each headline
[x for x in sample]

['Finding an Expansive View  of a Forgotten People in Niger',
 'And Now,  the Dreaded Trump Curse',
 'Venezuela’s Descent Into Dictatorship',
 'Stain Permeates Basketball Blue Blood',
 'Taking Things for Granted',
 'The Caged Beast Awakens',
 'An Ever-Unfolding Story',
 'O’Reilly Thrives as Settlements Add Up',
 'Mouse Infestation',
 'Divide in G.O.P. Now Threatens Trump Tax Plan']

In [ ]:
# Nested comprehesion for iteration over each element
[y for x in sample[0] for y in x]

['F',
 'i',
 'n',
 'd',
 'i',
 'n',
 'g',
 ' ',
 'a',
 'n',
 ' ',
 'E',
 'x',
 'p',
 'a',
 'n',
 's',
 'i',
 'v',
 'e',
 ' ',
 'V',
 'i',
 'e',
 'w',
 ' ',
 ' ',
 'o',
 'f',
 ' ',
 'a',
 ' ',
 'F',
 'o',
 'r',
 'g',
 'o',
 't',
 't',
 'e',
 'n',
 ' ',
 'P',
 'e',
 'o',
 'p',
 'l',
 'e',
 ' ',
 'i',
 'n',
 ' ',
 'N',
 'i',
 'g',
 'e',
 'r']

In [ ]:
def cleaning(data):
    cleaned = "".join(x for x in data if x not in string.punctuation).lower()
    cleaned = cleaned.encode("utf8").decode("ascii", "ignore")
    return cleaned

cleaned_l = [cleaning(x) for x in headlines]
cleaned_l[:10]

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'oreilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan']

Tokenization using Keras


Dataset preprocessing

TOKENIZATION

All the words and their indices will be stored in a dictionary which you can access it using tokenizer.word_index. Therefore, you can find the number of the unique words based on the number of elements in this dictionary:

num_words = len(tokenizer.word_index) + 1

That + 1 is because of reserving padding (i.e. index zero).


In [ ]:
sample = cleaned_l[:2]
sample

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse']

In [ ]:
tk = Tokenizer()
tk.fit_on_texts(sample)
tk.word_index

{'finding': 1,
 'an': 2,
 'expansive': 3,
 'view': 4,
 'of': 5,
 'a': 6,
 'forgotten': 7,
 'people': 8,
 'in': 9,
 'niger': 10,
 'and': 11,
 'now': 12,
 'the': 13,
 'dreaded': 14,
 'trump': 15,
 'curse': 16}

In [ ]:
tk.texts_to_sequences([sample[0]])

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]

We can see that the word_index does not have 0, which is reserved for padding

In [ ]:
tk.word_index

{'finding': 1,
 'an': 2,
 'expansive': 3,
 'view': 4,
 'of': 5,
 'a': 6,
 'forgotten': 7,
 'people': 8,
 'in': 9,
 'niger': 10,
 'and': 11,
 'now': 12,
 'the': 13,
 'dreaded': 14,
 'trump': 15,
 'curse': 16}

In [ ]:
len(tk.word_index)

16

In [ ]:
sample = cleaned_l[:5]
sample

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted']

In [ ]:
for line in sample:
    print(line[0])

f
a
v
s
t


In [ ]:
tk = Tokenizer()

def text_tokens(data):
    tk.fit_on_texts(data)
    # Our vocabulary size will be +1 due to incorporation of padding, which is treated like another word
    total_vocab = len(tk.word_index)+1
    input_sequences=[]
    
    for record in data:
        # This will give a 2D array, the first element will be the token list
        token_list = tk.texts_to_sequences([record])[0]
        for i in range(1, len(token_list)):
            # Append first two elements, then 3, then 4, then 5
            n_gram = token_list[:i+1]
            input_sequences.append(n_gram)
    return input_sequences, total_vocab

In [ ]:
inp_seq, vocab_size = text_tokens(cleaned_l)
inp_seq[:10]

[[391, 17],
 [391, 17, 5166],
 [391, 17, 5166, 523],
 [391, 17, 5166, 523, 4],
 [391, 17, 5166, 523, 4, 2],
 [391, 17, 5166, 523, 4, 2, 1601],
 [391, 17, 5166, 523, 4, 2, 1601, 134],
 [391, 17, 5166, 523, 4, 2, 1601, 134, 5],
 [391, 17, 5166, 523, 4, 2, 1601, 134, 5, 1951],
 [7, 57]]

Padding sequences to obtain complete data


Padding sequence

A uniform length of all the input vectors is created, that can be passed in the input layer of our LSTM model. We will also separate out the predictor features from the target labels.

The features will be all values from the start, till the 2nd last value The target will be the final value in the given sequence


In [ ]:
s = pd.Series(inp_seq)

In [ ]:
max_length = np.max(s.apply(lambda x: len(x)))
max_length

24

Now, we proceed to create a uniform length of all the input vectors, that can be passed in the input layer of our LSTM model. We will also separate out the predictor features from the target labels.

The features will be all values from the start, till the 2nd last value
The target will be the final value in the given sequence

In [ ]:
def padding(inp):
    max_seq_len = max_length
    inp_seq = np.array(pad_sequences(inp, maxlen=max_seq_len, padding='pre'))
    features, target = inp_seq[:, :-1], inp_seq[:, -1]
    target = ku.to_categorical(target, num_classes=vocab_size)
    return features, target, max_seq_len

features, target, max_seq_len = padding(inp_seq)

In [ ]:
features.shape

(51770, 23)

In [ ]:
target.shape

(51770, 11265)


RNN Model

LSTMs have an additional state called ‘cell state’ through which the network makes adjustments in the information flow. The advantage of this state is that the model can remember or forget the leanings more selectively. To learn more about LSTMs, here is a great post. Lets architecture a LSTM model in our code. I have added total three layers in the model.

Input Layer : Takes the sequence of words as input LSTM Layer : Computes the output using LSTM units. I have added 100 units in the layer, but this number can be fine tuned later. Dropout Layer : A regularisation layer which randomly turns-off the activations of some neurons in the LSTM layer. It helps in preventing over fitting. (Optional Layer) Output Layer : Computes the probability of the best possible next word as output We will run this model for total 100 epoochs but it can be experimented further.


In [ ]:
model=Sequential()
# We define max length - 1 as now our features have one column less (used as target label)
model.add(Embedding(vocab_size, 10, input_length=max_seq_len-1))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 10)            112650    
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 11265)             1137765   
                                                                 
Total params: 1,294,815
Trainable params: 1,294,815
Non-trainable params: 0
_________________________________________________________________



Train the Model

In [ ]:
model.fit(features, target, epochs=30, verbose=5)

Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


Prediction function

In [ ]:
def predictions(seed_text, next_words, model, max_seq):
    for _ in range(next_words):
        # Conver the seed text into tokens
        token_list = tk.texts_to_sequences([seed_text])[0]
        # Pad the seed text with seq-1 (as we only have features)
        token_list = pad_sequences([token_list], maxlen=max_seq-1, padding='pre')
        # Predict the class of the next word
        predicted = np.argmax(model.predict(token_list),axis=1)        
        
        outputs = ""
        # Iterate over the dictionary, to get the word
        for word, index in tk.word_index.items():
            if index==predicted:
                output_word = word
                break
        # Add the word in the seed text, for the next prediction
        seed_text = seed_text+" "+output_word
    return seed_text.title()


In [ ]:
#predicted = np.argmax(model.predict(token_list),axis=1)
print (predictions("united states", 5, model, max_seq_len))
print (predictions("preident trump", 4, model, max_seq_len))
print (predictions("donald trump", 4, model, max_seq_len))
print (predictions("india and china", 4, model, max_seq_len))
print (predictions("new york", 4, model, max_seq_len))
print (predictions("science and technology", 5, model, max_seq_len))

1/1 [==============================] - 0s 103ms/step
United States Marred As Legal Carry On
1/1 [==============================] - 0s 68ms/step
Preident Trump Is A Solution For
1/1 [==============================] - 0s 94ms/step
Donald Trump Vs The Upper Upper
1/1 [==============================] - 0s 29ms/step
India And China And Shift Over Border
1/1 [==============================] - 0s 19ms/step
New York Today A Belated Trade
1/1 [==============================] - 0s 20ms/step
Science And Technology Grip The Disability Search Of
